# ** Today's agenda:
(1) Hyper parameter tuning in our ML project
(2) How to use generator functions to load data by chunks?
     - When a data file is moderately large and the Python script freezes, what technique should we use??
     - Use the LazyPredic Python script as an example
(3) We have a random signal. We want to find...
   a.  How many times it relaxed?
   b. When was the first time it relaxed?
   c. When was the last time it relaxed?
   d. What is its relaxation time on each occasion?
   e. What is its average relaxation time?

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('/content/drive/MyDrive/full.regular.txt', delim_whitespace=True)

In [4]:
data.describe()

,0,0.000,0.000.1,0.000.2,1,1.1,1.2,1.3,1.4,0.1
count,1.079133e+06,1.079133e+06,1.079133e+06,1.079133e+06,1.079133e+06,1.079133e+06,1.079133e+06,1.079133e+06,1.079133e+06,1079133.0
mean,1.502021e+02,5.965603e+00,7.416875e+00,9.035807e+00,1.504170e-02,7.553749e-02,2.246748e-01,8.234490e-01,2.491768e+00,0.0
std,1.267608e+02,8.436803e-01,2.182458e+00,3.029510e+00,1.221747e-01,2.767069e-01,4.954270e-01,1.201071e+00,2.386876e+00,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,5.500000e+01,5.467000e+00,5.278000e+00,6.272000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.0
50%,1.180000e+02,5.795000e+00,7.869000e+00,8.905000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.0
75%,2.140000e+02,6.563000e+00,9.370000e+00,1.184800e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,4.000000e+00,0.0
max,1.015000e+03,7.826000e+00,1.159000e+01,1.492200e+01,2.000000e+00,4.000000e+00,6.000000e+00,1.000000e+01,1.800000e+01,0.0


In [5]:
y = data.iloc[:, 2]
X = data.iloc[:, 3:11]

In [11]:
print(X.shape)

(1079133, 8)


In [8]:
X_samll = X.iloc[0:,0:]
y_samll = y.iloc[0:]

In [9]:
y.value_counts()

C
H    459325
C    377953
E    241855
Name: count, dtype: int64

In [10]:
target_encoding = []
for x in y_samll:
  if x == "H":
    target_encoding.append(0)
  elif x == "C":
    target_encoding.append(1)
  else:
      target_encoding.append(2)
target_encoding = np.array(target_encoding)

In [12]:
from imblearn.under_sampling import NearMiss
nm1 = NearMiss(version=1)
X_resampled_nm1, y_resampled = nm1.fit_resample(X, target_encoding)

In [ ]:
from imblearn.under_sampling import NearMiss
nm1 = NearMiss(version=1)
X_resampled_nm1, y_resampled = nm1.fit_resample(X, target_encoding)

In [13]:
print(X_resampled_nm1.shape)
print(y_resampled.shape)

(725565, 8)
(725565,)


In [14]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_rus_resampled, y_rus_resampled = rus.fit_resample(X_resampled_nm1, y_resampled)

In [15]:
print(X_rus_resampled.shape)
print(y_rus_resampled.shape)

(725565, 8)
(725565,)


In [36]:
def chunck_generation(x_i,y_i, segmentation):
  for i in range(segmentation):
    X = x_i[0:int((x_i.shape[0])/segmentation)]
    Y = y_i[0:int((y_i.shape[0])/segmentation)]
  return   X, Y


In [37]:
X_chunck_out,y_chunck_out  = chunck_generation(X_samll,y_samll, 100)

In [38]:
X_chunck_out.shape

(10791, 8)

In [39]:
y_chunck_out.shape

(10791,)

In [32]:
!pip install lazypredict

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
from lazypredict.Supervised import LazyClassifier
X_train, X_test, y_train, y_test = train_test_split(X_chunck_out, y_chunck_out,test_size=.3,random_state =123)

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

 90%|████████▉ | 26/29 [00:30<00:03,  1.09s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001451 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 807
[LightGBM] [Info] Number of data points in the train set: 7553, number of used features: 8
[LightGBM] [Info] Start training from score -1.039118
[LightGBM] [Info] Start training from score -1.507326
[LightGBM] [Info] Start training from score -0.856297


100%|██████████| 29/29 [00:31<00:00,  1.07s/it]


In [42]:
print(models)

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
RandomForestClassifier             0.79               0.79    None      0.79   
LGBMClassifier                     0.79               0.79    None      0.79   
ExtraTreesClassifier               0.78               0.78    None      0.78   
SVC                                0.77               0.76    None      0.77   
KNeighborsClassifier               0.76               0.76    None      0.76   
LabelSpreading                     0.76               0.75    None      0.76   
BaggingClassifier                  0.76               0.75    None      0.77   
LabelPropagation                   0.76               0.75    None      0.76   
NuSVC                              0.75               0.75    None      0.75   
AdaBoostClassifier                 0.75               0.74    None      0.75   
LogisticRegression                 0.74 